# [SITCOM-2038] MTDome Telemetry Latency Evaluation

[SITCOM-2038]: https://rubinobs.atlassian.net/browse/SITCOM-2038

In [ ]:
# User input 
start_day_obs = 20250410
end_day_obs = 20250410

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np

from astropy.time import Time
from pathlib import Path

from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData, getDayObsEndTime, getDayObsStartTime

## Query Data Sets

In [ ]:
def get_colorblind_complements(base_color):
    """
    Generate three colorblind-safe complementary colors based on base_color.
    Returns a list of hex colors.
    """
    # Convert to HSV
    hsv = mcolors.rgb_to_hsv(mcolors.to_rgb(base_color))

    # Shift hue for complements (120°, 180°, 240° in degrees → in [0, 1] it's 1/3, 0.5, 2/3)
    hues = [(hsv[0] + offset) % 1.0 for offset in (1/3, 0.5, 2/3)]
    
    # Keep saturation and value modest for readability
    sv = (max(0.5, hsv[1]), max(0.5, hsv[2]))
    complements = [mcolors.hsv_to_rgb((h, *sv)) for h in hues]

    return [mcolors.to_hex(c) for c in complements]


In [ ]:
def query_azimuth_timestamps(_start_day_obs, _end_day_obs, topic):
    """Query the data from the EFD. We just need the timestamps from a
    telemetry. The columns do not matter.

    Parameters
    ----------
    _day_obs : int
        Day Obs in YYYYMMDD representation.

    Returns
    -------
    _df : pd.DataFrame
        A table containing the data and the timestamps.
    """
    start_time = getDayObsStartTime(_start_day_obs)
    end_time = getDayObsEndTime(_end_day_obs)
    
    _df = getEfdData(
        client=efd_client,
        topic=topic,
        columns=["timestamp"],
        begin=start_time,
        end= end_time
    )

    # Convert TAI to UTC in isot format
    t = Time(_df["timestamp"].to_numpy(), format="unix_tai", scale="tai")
    _df["timestamp"] = t.utc.isot
     
    print(
        f"Queried {topic} from"
        f"  {start_time} to {end_time}"
    )
    
    return _df.dropna()

In [ ]:
# Set path for plots
plot_path = Path("./plots")
plot_path.mkdir(exist_ok=True, parents=True)

# Base and complementary colors
base_color = "#058B8C"
colors = get_colorblind_complements(base_color)

# Set global font size for labels, titles, and ticks
plt.rcParams.update({
    "axes.grid": True,
    "axes.labelsize": 12,  
    "axes.titlesize": 14,
    "axes.formatter.useoffset": False,
    "axes.formatter.use_mathtext": False,
    "axes.formatter.limits": (-100, 100),
    "axes.prop_cycle": plt.cycler(color=[base_color] + colors),
    "figure.figsize": (11, 6),
    "font.size": 12,
    "grid.color": "#b0b0b0", 
    "grid.linestyle": ":",
    "grid.linewidth": 0.5,
    "grid.alpha": 0.75,
    "xtick.labelsize": 12,  
    "ytick.labelsize": 12,  
})

efd_client = makeEfdClient()

In [ ]:
df_azimuth = query_azimuth_timestamps(start_day_obs, end_day_obs, "lsst.sal.MTDome.azimuth")
df_shutter = query_azimuth_timestamps(start_day_obs, end_day_obs, "lsst.sal.MTDome.apertureShutter")

## Plot Timeline

In [ ]:
def plot_timeline_index(_df, _ax, _label, color="#058B8C"):
    """
    Plots a timeline using the DataFrame index (assumed to be UTC).
    """
    t = _df.timestamp.index.dropna()
    delta_t = t[1:] - t[:-1]

    mask = delta_t.seconds < 60
    delta_t = delta_t[mask].total_seconds().to_numpy()

    index = t[1:]
    index = index[mask]

    _ax.semilogy(index, delta_t, color=color, marker=".", linestyle="", alpha=0.25, label=_label)
    _ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
    return _ax


def plot_timeline_timestamp(_df, _ax, _label, color="#058B8C"):
    """
    Plots a timeline using the 'timestamp' column (assumed to be in UTC ISOT).
    """
    t = pd.to_datetime(_df.timestamp.dropna())
    delta_t = t[1:].to_numpy() - t[:-1].to_numpy()

    mask = delta_t < np.timedelta64(60, 's')
    delta_t = delta_t[mask] / np.timedelta64(1, 's')
    index = t[1:][mask]

    _ax.semilogy(index, delta_t, color=color, marker="x", linestyle="", alpha=0.25)
    _ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
    
    print(
        f"Average delta_t: {delta_t.mean()}\n"
        f"Min delta_t: {delta_t.min()}\n"
        f"Max delta_t: {delta_t.max()}\n"
        f"Std delta_t: {delta_t.std()}\n"
    )

    return _ax

In [ ]:
fig_name = "SITCOM-2055 Timeline"

fig, (ax1, ax2) = plt.subplots(num=fig_name, nrows=2, ncols=2, sharex=True)
color = "#058B8C"
colors = get_colorblind_complements(color)

ax1[0] = plot_timeline_index(df_azimuth, ax1[0], "Azimuth Telemetry", color="C0")
ax1[0].set_ylabel("Az EFD\nDelta T (s)")

print("Azimuth Telemetry:")
ax1[1] = plot_timeline_timestamp(df_azimuth, ax1[1], "Azimuth Telemetry", color="C1")
ax1[1].yaxis.set_label_position("right")
ax1[1].yaxis.tick_right()
ax1[1].set_ylabel("Az TAI\nDelta T (s)")

ax2[0] = plot_timeline_index(df_shutter, ax2[0], "Shutter Telemetry", color="C2")
ax2[0].set_ylabel("Shutter EFD\nDelta T (s)")
ax2[0].set_xlabel("Time [UTC]")

print("Shutter Telemetry:")
ax2[1] = plot_timeline_timestamp(df_shutter, ax2[1], "Shutter Telemetry", color="C3")
ax2[1].yaxis.set_label_position("right")
ax2[1].yaxis.tick_right()
ax2[1].set_ylabel("Shutter TAI\nDelta T (s)")
ax2[1].set_xlabel("Time [UTC]")

fig.suptitle(f"{fig_name}\nDome telemetry timeline from {start_day_obs} to {end_day_obs}")
fig.autofmt_xdate()
fig.tight_layout(h_pad=0, w_pad=0)

plt.savefig(plot_path / f"SITCOM-2038 Timeline - {start_day_obs} to {end_day_obs}.png")
plt.show()

## Plot Histogram

In [ ]:
def plot_hist_index(_df, _ax, color="#058B8C", log=False):
    t = _df.timestamp.index.dropna()
    delta_t = t[1:] - t[:-1]
    mask = delta_t.seconds < 60
    delta_t = delta_t[mask].total_seconds().to_numpy()

    _ax.hist(delta_t, bins=30, color=color, edgecolor="white", linewidth=0.75, log=log)
    _ax.set_xlim(0, 2)
    return _ax


def plot_hist_timestamp(_df, _ax, color="#058B8C", log=False):
    t = pd.to_datetime(_df.timestamp.dropna())
    delta_t = t[1:].to_numpy() - t[:-1].to_numpy()
    mask = delta_t < np.timedelta64(60, 's')
    delta_t = delta_t[mask] / np.timedelta64(1, 's')

    _ax.hist(delta_t, bins=30, color=color, edgecolor="white", linewidth=0.75, log=log)
    _ax.set_xlim(0, 2)
    return _ax


In [ ]:
fig_name = "SITCOM-2055 Histogram"

fig, (ax1, ax2) = plt.subplots(num=fig_name, ncols=2, nrows=2, sharex=True, figsize=(9,6))

ax1[0] = plot_hist_index(df_azimuth, ax1[0], color="C0")
ax1[0].set_ylabel("Az EFD\nN events")

ax1[1] = plot_hist_timestamp(df_azimuth, ax1[1], color="C1")
ax1[1].yaxis.set_label_position("right")
ax1[1].yaxis.tick_right()
ax1[1].set_ylabel("Az TAI\nN events")
ax1[1].set_xlabel("Delta t [s]")

ax2[0] = plot_hist_index(df_shutter, ax2[0], color="C2")
ax2[0].set_ylabel("Shutter EFD\nN events")

ax2[1] = plot_hist_timestamp(df_shutter, ax2[1], color="C3")
ax2[1].yaxis.set_label_position("right")
ax2[1].yaxis.tick_right()
ax2[1].set_ylabel("Shutter TAI\nN events")
ax2[1].set_xlabel("Delta t [s]")

fig.suptitle(f"{fig_name}\nDome telemetry histograms from {start_day_obs} to {end_day_obs}")
fig.tight_layout(h_pad=0, w_pad=0)
plt.savefig(plot_path / f"SITCOM-2055 Histogram - {start_day_obs} to {end_day_obs}.png")
plt.show()